In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gym

%matplotlib inline

In [2]:
env = gym.make('FrozenLake-v0')
episodes = 5000000
reward_history = []
EPSILON = 0.3
GAMMA = 0.9
render = False

In [3]:
def max_dict(d):
    # returns the argmax (key) and max (value) from a dictionary   
    max_key = None
    max_val = float('-inf')
    for k, v in d.items():
        if v > max_val:
            max_val = v
            max_key = k
    return max_key, max_val

In [4]:
def moving_average(values, n=100) :
    ret = np.cumsum(values, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [5]:
def epsilon_action(a, eps=0.1):
    p = np.random.random()
    if p < (1 - eps):
        return a
    else:
        return env.action_space.sample()

In [6]:
def calculate_returns(states_actions_rewards):
    # calculate the returns by working backwards from the terminal state
    G = 0
    states_actions_returns = []
    first = True
    for s, a, r in reversed(states_actions_rewards):
        # a terminal state has a value of 0 by definition
        # this is the first state we encounter in the reversed list
        # we'll ignore its return (G) since it doesn't correspond to any move
        if first:
            first = False
        else:
            states_actions_returns.append((s, a, G))
        G = r + GAMMA*G
    states_actions_returns.reverse() # back to the original order of states visited

    return states_actions_returns

In [7]:
def main():
    
    eps = EPSILON
    R = {}
    Q = {}
    P = {}
    deltas = []
    rewards_list = []
    
    for s in range(env.observation_space.n):
        P[s] = env.action_space.sample()
        Q[s] = {}
        for a in range(env.action_space.n):
            Q[s][a] = 0
            R[(s,a)] = []
            
    print("Initial policy values: {} \n\n".format(P))         
        
    for i_episode in range(episodes):            
        
        # Play episode
        state = env.reset()        
        reward_per_episode = 0  
        states_actions_returns = []
        action = epsilon_action(P[state], eps)
        states_actions_rewards = [(state, action, 0)]
        for t in range(10000):  # Don't infinite loop while learning                           
            state, reward, done, _ = env.step(action) 
            reward_per_episode += reward
            if render:
                env.render()                           
            if done:
                states_actions_rewards.append((state, None, reward))
                break
            else:
                action = epsilon_action(P[state], eps)
                states_actions_rewards.append((state, action, reward))    
        rewards_list.append(reward_per_episode)        
        states_actions_returns = calculate_returns(states_actions_rewards)        
        
        # calculate Q(s,a)
        biggest_change = 0
        seen_state_action_pairs = set()
        for s, a, r in states_actions_returns:
            # check if we have already seen s
            # first-visit Monte Carlo optimization
            sa = (s, a)
            if sa not in seen_state_action_pairs:
                R[sa].append(r)
                old_q = Q[s][a]
                # the new Q[s][a] is the sample mean of all our returns for that (state, action)
                Q[s][a] = np.mean(R[sa])
                biggest_change = max(biggest_change, np.abs(old_q - Q[s][a]))
                seen_state_action_pairs.add(sa)
        deltas.append(biggest_change)
        
        # calculate new policy pi(s) = argmax[a]{ Q(s,a) }
        for s in P.keys():
            a, _ = max_dict(Q[s])
            P[s] = a
            
        avg_reward = np.mean(rewards_list[-500:])     
        if i_episode % 5000 == 0:            
            avg_delta = np.mean(deltas[-500:])  
            print ("Episode: {}   Avg reward: {:3.3f}  Avg Delta: {:8.8f}  Epsilon: {:3.3f}".format(
                i_episode, avg_reward, avg_delta, eps))
            eps = max(eps - 0.005, 0.0)
            
        if len(rewards_list) > 10000:
            rewards_list.pop(0)
            
        if len(deltas) > 10000:
            deltas.pop(0)
          
        if avg_reward >= env.spec.reward_threshold: 
            print("########## Solved! ###########")
            break
            
  
    # calculate values for each state (just to print and compare)
    # V(s) = max[a]{ Q(s,a) }
    V = {}
    for s in P.keys():
        V[s] = max_dict(Q[s])[1]
        
    print("\nV values: {}".format(V))    
   
    return P
            
print("action_space={}".format(env.action_space))
print("obs_space={}".format(env.observation_space))
print("threshold={} \n".format(env.spec.reward_threshold))
policy = main()

print("Policy values: {}".format(policy)) 

action_space=Discrete(4)
obs_space=Discrete(16)
threshold=0.78 

Initial policy values: {0: 0, 1: 3, 2: 1, 3: 0, 4: 3, 5: 3, 6: 3, 7: 3, 8: 1, 9: 3, 10: 1, 11: 2, 12: 0, 13: 3, 14: 2, 15: 0} 


Episode: 0   Avg reward: 0.000  Avg Delta: 0.00000000  Epsilon: 0.300
Episode: 5000   Avg reward: 0.072  Avg Delta: 0.00100847  Epsilon: 0.295
Episode: 10000   Avg reward: 0.056  Avg Delta: 0.00029844  Epsilon: 0.290
Episode: 15000   Avg reward: 0.060  Avg Delta: 0.00022527  Epsilon: 0.285
Episode: 20000   Avg reward: 0.078  Avg Delta: 0.00016754  Epsilon: 0.280
Episode: 25000   Avg reward: 0.084  Avg Delta: 0.00014455  Epsilon: 0.275
Episode: 30000   Avg reward: 0.100  Avg Delta: 0.00011969  Epsilon: 0.270
Episode: 35000   Avg reward: 0.098  Avg Delta: 0.00010169  Epsilon: 0.265
Episode: 40000   Avg reward: 0.156  Avg Delta: 0.00012559  Epsilon: 0.260
Episode: 45000   Avg reward: 0.148  Avg Delta: 0.00009544  Epsilon: 0.255
Episode: 50000   Avg reward: 0.164  Avg Delta: 0.00009022  Epsilon: 0.2

Episode: 535000   Avg reward: 0.510  Avg Delta: 0.00000152  Epsilon: 0.000
Episode: 540000   Avg reward: 0.538  Avg Delta: 0.00000145  Epsilon: 0.000
Episode: 545000   Avg reward: 0.532  Avg Delta: 0.00000147  Epsilon: 0.000
Episode: 550000   Avg reward: 0.504  Avg Delta: 0.00000139  Epsilon: 0.000


KeyboardInterrupt: 

In [ ]:
def test(policy):
    print (policy)
    state = env.reset()        
    action = epsilon_action(policy[state], 0)
    for t in range(10000):                        
        state, reward, done, _ = env.step(action) 
        env.render()                           
        if done:
            print("Reward: {}".format(reward))
            break

test(policy)
        